In [45]:
from pathlib import Path

import pyrosm

import src.data.mapillary as mapi
import src.data.osm as osmd
#from src.data.mapillary import download_mapillary_image_by_key, download_mapillary_object_detection_by_key
#from src.data.osm import add_mapillary_key_to_network
from ipyleaflet import Map, Marker, Icon, Polyline
from importlib import reload

### Settings

In [2]:
input_filepath = "../data/raw/berlin-latest.osm.pbf"
min_quality_score = 4
output_dir = "../data/processed/out"

### Code (see make_dataset.py)

In [3]:
osm = pyrosm.OSM(input_filepath)

In [4]:
network = osm.get_network(
    network_type="cycling",
    extra_attributes=["surface", "smoothness"]
)

In [5]:
network = network.head(100)

In [6]:
# Filter relevant columns
network = network[["id", "geometry", "surface", "smoothness"]]

In [7]:
# Filter only records where both surface and smoothness is set
network = network[(~network["surface"].isna()) & (~network["smoothness"].isna())]

In [71]:
# Get Mapillary keys for each street
street_mapillary_df = osmd.add_mapillary_key_to_network(network.head(10), min_quality_score)
street_mapillary_df

Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.525808394391273,52.48913043665209,13.527869293873632,52.49044905555783&per_page=500&sort_by=key&min_quality_score=4
DONE
Total images: 167
Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.523868068725816,52.482789917870114,13.525887018854531,52.489174160275965&per_page=500&sort_by=key&min_quality_score=4
DONE
Total images: 270
Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.635509405757002,52.36593718240352,13.637040327791425,52.369278836433104&per_page=500&sort_by=key&min_quality_score=4
DONE
Total images: 122
Request URL: https://a.mapillary.com/v3/images?client_id=TmxURHBKRFFFRTc1dWZscGpLWW5VUDo1ZjYyYmFjYjY5MTA3MDNk&bbox=13.628494275759923,52.350349756347946,13.631994532529342,52.35217994781752&per_page=500&sort_by=key&min_quality

,geometry,id,img_coords,mapillary_key,smoothness,surface
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5276836 52.4903206),0BdKIR4sPdlLiabOfAmD6B,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5277531 52.4903519),Gv8rUSeGkmDzs0yk1OtsYj,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5277539 52.4903672),bIfWS4heXtCoVDddCMPWi7,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5277553 52.4903847),GJAfy1dIDRs3mR78ddlEh2,bad,sett
2,"MULTILINESTRING ((13.52783 52.49043, 13.52639 ...",4045220.0,POINT (13.5277573 52.4903987),nLFpoV5wWGAZWFn6kpWOTh,bad,sett
...,...,...,...,...,...,...
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,POINT (13.6376074 52.3701573),4HztkqBrTqYNyLhPvuvElA,bad,cobblestone
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,POINT (13.6375555 52.3700871),ZXBFAEEofR8vA8KUcTNNTT,bad,cobblestone
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,POINT (13.6375275 52.3700385),J9U6Q30u2OxkZz5x9Xg3Fj,bad,cobblestone
51,"MULTILINESTRING ((13.63700 52.36926, 13.63713 ...",4068154.0,POINT (13.6411976 52.3756009),SfbKzJ-H4u2y4uZ6QHL9SA,bad,cobblestone


In [29]:
# Create output dir
Path(f"{output_dir}").mkdir(parents=True, exist_ok=True)

In [30]:
# Create sub dirs
image_dir = f"{output_dir}/images"
object_detection_dir = f"{output_dir}/object_detections"

Path(image_dir).mkdir(exist_ok=True)
Path(object_detection_dir).mkdir(exist_ok=True)

In [31]:
# Export street_mapillary_df as data.csv
street_mapillary_df.to_csv(f"{output_dir}/data.csv", index=False)

In [72]:
# Download images and object detections
for _, row in street_mapillary_df.iterrows():
    mapi.download_mapillary_image_by_key(row["mapillary_key"], download_dir=image_dir)
    #download_mapillary_object_detection_by_key(row["mapillary_key"],
                                               #download_dir=object_detection_dir)

In [73]:
map_center = street_mapillary_df.iloc[0].img_coords
map_center = [map_center.y, map_center.x]

street_id_added = list()
m = Map(zoom=16, center=map_center, max_zoom=22)
for _, row in street_mapillary_df.iterrows():
    img_path = Path(output_dir) / "images" / (row.mapillary_key + ".jpg")
    assert img_path.is_file(), img_path
    icon = Icon(icon_url=str(img_path), 
                icon_size=[95, 38])
    marker_center = [row.img_coords.y, row.img_coords.x]
    marker = Marker(location=marker_center, draggable=False, icon=icon)
    m.add_layer(marker)
    
    if row.id not in street_id_added:
        street_id_added.append(row.id)
        line_shapely = row.geometry
        line_map_locactions = list()

        for single_line in line_shapely.geoms:
            line_map_locactions.extend([reversed(p) for p in single_line.coords])
        
        line = Polyline(
            locations=line_map_locactions,
            color="green" ,
            fill=False
        )
        m.add_layer(line)
m

Map(center=[52.4903206, 13.5276836], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…